In [1]:
# use docker to parse grib file
#docker pull osgeo/gdal
#docker run --rm -v "C:/Users/WeilunShi/OneDrive - CGIAR/Desktop/Food Crisis and Insecurity/0.external_data/ERA5_data:/data" osgeo/gdal:latest gdal_translate -of GTiff /data/ERA_2.grib /data/output_2.tif

import numpy as np
import rasterio
import geopandas as gpd
from rasterio.features import shapes
from shapely.geometry import shape

# Read the .tif file with Rasterio
with rasterio.open(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\0.external_data\ERA5_soil_water\output_2.tif') as src:
    image = src.read(1)  # Read the first band
    transform = src.transform

# Convert the image data type to int16
image = image.astype(np.int16)
# Extract shapes from the raster data
shapes_and_values = list(shapes(image, transform=transform))
geometries = [shape(s[0]) for s in shapes_and_values]
values = [s[1] for s in shapes_and_values]

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame({'geometry': geometries, 'value': values})

# Set the Coordinate Reference System (CRS) to match the input raster
gdf.crs = src.crs


# set CRS to 4236
gdf = gdf.to_crs(epsg=4326)



# read geojson
areas = gpd.read_file(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\\2.merge_data\intermediate\output10.geojson')


# spatial join
result = gpd.sjoin(areas, gdf, how='left', op='intersects')

# rename
result = result.rename(columns={'value': 'soil_water'})

#parse missing value
result.loc[result['soil_water']==9999] = np.nan

# summarize
results = result.groupby(['date','title'])['soil_water'].mean().reset_index()

# merge
results = results.merge(result, on=['title','date'], how='right')



results.drop(columns=['soil_water_y','index_right'], inplace=True)
#rename
results.rename(columns={'soil_water_x':'soil_water'}, inplace=True)


#drop duplicates
results.drop_duplicates(inplace=True)

# convert dataframe to geodataframe
results = gpd.GeoDataFrame(results, geometry='geometry')

# save as geojson
results.to_file(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\\2.merge_data\intermediate\output11.geojson', driver='GeoJSON')

c:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
